In [1]:
#!pip install tweepy
import tweepy 
from tweepy import Stream
from tweepy.streaming import StreamListener 
import json
import time
import datetime

## Predefined some variables that needed for scraping

In [2]:
# Token defined
ACCESS_TOKEN = '1017116258-GUN3YAWrElxuVDDkeazQui0rhOsuR08UNgIRYg2'
ACCESS_TOKEN_SECRET = 'My6igTJbpw1nHtNwid3P9qksNicScwLxTlGrkFcDuTK53'
CONSUMER_KEY = '0YJQlQtmUf0Y6N7NVu5rA33fk'
CONSUMER_SECRET = 'bduF2iguZjQSkq6G17jsX0yax9ICOcLWNYUicR99b51GLzeNam'

# Keywords that needed for scraping
text_list = ['PvdA', 'Partij van de Arbeid', 'Lodewijk Asscher', 'Marleen Barth', 'Moorlag', 'Aboutaleb', 
             'Arib', 'VVD', 'Volkspartij voor Vrijheid en Democratie', 'Mark Rutte', 'Eric Wiebes', 
             'Klaas Dijkhoff', 'Halbe Zijlstra', 'PVV',  'Partij van de Vrijheid', 'Geert Wilders', 
             'Martin Bosma', 'DENK', 'Tunahan Kuzu','D66', 'Democraten 66', 'Kajsa Ollongren', 
             'Alexander Pechtold', 'Jan Paternotte', 'Ingrid van Engelshoven', 'Wouter Koolmees', 
             'Sigrid Kaag', 'Stientje van Veldhoven', 'FvD', 'Forum voor Democratie', 'Forum', 
             'Thierry Baudet', 'Theo Hiddema', 'Yernaz Ramautarsing', 'Annabel Nanninga', 'CDA', 
             'Christendemocraten', 'Christendemocratie', 'Buma', 'Sybrand van Haersma-Buma', 'Mona Keizer', 
             'Wopke Hoekstra', 'Hugo de Jonge', 'Ferdinand Grapperhaus', 'GroenLinks', 'Jesse Klaver',
             'ChristenUnie', 'Arie Slob', 'Carola Schouten','SGP', 'Kees van der Staaij', 'Partij voor de Dieren', 
             'Marianne Thieme', 'SP','Socialistische Partij', 'Lillian Marijnissen', 'Arjan Vliegenthart', 
             'Leefbaar Rotterdam', 'Joost Eerdmans', 'NIDA', 'Cemil Yildaz', 'Femke Halsema', '50plus', 'Henk Krol', 
             'NIDA', 'Cemil Yildaz', 'Groep de Mos', 'Richard de Mos', 'Rachid Guernaoui', 'Janice Roopram', 
             'Haagse stadspartij', 'Peter Bos', 'Joris Wijsmuller', 'Fatima Faid','Islam Democraten', 'Hasan Küçük', 
             'Arnoud van Doorn', 'Boudewijn Revis', 'Judith Oudshoorn-van Ginderen', 'Robert van Asten', 
             'Hanneke van der Werf', 'Dennis Groenewold', 'Arjen Kapteijns', 'Erlijn Wenink', 'Karen Gerbrands', 
             'Elias van Hees','Martijn Balster', 'Mikal Tseggai', 'Karsten Klein', 'Danielle Koster','Christine Teunissen', 
             'SP', 'Bart van Kent', 'Aisha Akhiat', 'Pieter Grinwis'] 

# Hastags keywords that needed for scraping
hashtag_list = ['#GR2018', '#Baudet', '#Pechtold', '#gr2018', '#hijzeihetecht', '#forumvoorfantasie', 
                '#GR18', '#FVD', '#D66', '#PVV', '#fvdemocratie', '#GR2018RT', '#FvD', '#PvdA', '#d66', 
                '#gemeenteraadsverkiezingen', '#GR2', '#gemeenteraadsverkiezingen2018', '#baudet', '#VVD', 
                '#fvd', '#hoedan', '#StemPVV', '#jinek', '#GroenLinks', '#gr2018RT', '#pvda','#partijkartel', 
                '#antisemitisme', '#buitenhof', '#Gemeenteraads', '#referendum', '#Gr2018', '#groenlinks', 
                '#pechtold', '#Jinek', '#Coentunnel', '#Asscher', '#CDA', '#GR20', '#LelystadAirport', 
                '#Buma', '#PvdAGroenLinks', '#leefbaarrotterdam', '#Ascher', '#demoniseren','#Kuzu', 
                '#lijsttrekkers', '#integratie', '#voltooidleven', '#campagne', '#GRV2018', '#Klaver', 
                '#nepnieuws', '#dwdd', '#asscher', '#klaver', '#cda', '#Gemeenteraadsverkiezingen', 
                '#klimaat', '#Marijnissen', '#ThierryBaudet', '#Rutte', '#hartvoordenhaag']


In [3]:

class MyListener(StreamListener):
    def __init__(self, api, file_name_base):
        self.tweet_list=[]
        self.count = 1
        self.file_name_base = file_name_base
       
    def on_data(self, data):
        try:
            j = json.loads(data)
            self.tweet_list.append(j)
            n = len(self.tweet_list)
  
            if (n % 100 == 0):
                ts = time.time()
                stamp = datetime.datetime.fromtimestamp(ts).strftime('%m%d%H%M%S')
                file_name = self.file_name_base + stamp + '.json'
                with open(file_name , 'w') as f:
                    json.dump(self.tweet_list, f)
                print("Output File", self.count)
                self.count = self.count + 1
                self.tweet_list = []
            return True
        except BaseException as e:
            print("Error on_data:", str(e))
        return True
 
    def on_error(self, status):
        print("Error",status)
        return False


In [4]:
class Credentials:
    def __init__(self, access_token, access_token_secret, consumer_key, 
        consumer_secret):
        self.access_token = access_token
        self.access_token_secret = access_token_secret
        self.consumer_key = consumer_key
        self.consumer_secret = consumer_secret

In [ ]:
def mine_tweets(credentials, search_terms, file_name):
    auth = tweepy.OAuthHandler(credentials.consumer_key, credentials.consumer_secret)
    auth.set_access_token(credentials.access_token, credentials.access_token_secret)

    api = tweepy.API(auth)

    twitter_stream = Stream(auth, MyListener(auth, file_name))

    print("Starting up!")

    while(True):
        try:
            twitter_stream.filter(track=search_terms)
        except:
            time.sleep(30)
            print("program restart")

In [ ]:
mentions_list = []
from_list = []
to_list = []
my_list = []

full_list = text_list
full_list.extend(hashtag_list)
full_list.extend(mentions_list)
full_list.extend(from_list)
full_list.extend(to_list)
full_list.extend(my_list)

credentials = Credentials(ACCESS_TOKEN, ACCESS_TOKEN_SECRET,
    CONSUMER_KEY, CONSUMER_SECRET)

file_name_base = 'twitter'

mine_tweets(credentials, full_list, file_name_base)

Starting up!
Output File 1
Output File 2
Output File 3
Output File 4
Output File 5
Output File 6
Output File 7
Output File 8
Output File 9
Output File 10
Output File 11
Output File 12
Output File 13
Output File 14
Output File 15
Output File 16
Output File 17
Output File 18
Output File 19
Output File 20
Output File 21
Output File 22
Output File 23
Output File 24
Output File 25
Output File 26
Output File 27
Output File 28
Output File 29
Output File 30
Output File 31
Output File 32
Output File 33
Output File 34
Output File 35
Output File 36
Output File 37
Output File 38
Output File 39
Output File 40
Output File 41
Output File 42
Output File 43
Output File 44
Output File 45
Output File 46
Output File 47
Output File 48
Output File 49
Output File 50
Output File 51
Output File 52
Output File 53
Output File 54
Output File 55
Output File 56
Output File 57
Output File 58
Output File 59
Output File 60
Output File 61
Output File 62
Output File 63
Output File 64
Output File 65
Output File 66
Output